<a href="https://colab.research.google.com/github/kcdheepak/ml-model/blob/main/AI_quality_checker_using_xg.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
!pip install xgboost scikit-learn pandas PyPDF2 joblib



In [7]:


import os
import zipfile
import PyPDF2
import pandas as pd
import xgboost as xgb
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from google.colab import files

uploaded = files.upload()

with zipfile.ZipFile("resume_samples.zip", 'r') as zip_ref:
    zip_ref.extractall("resumes")

texts = []
labels = []

for filename in os.listdir("resumes"):
    if filename.endswith('.pdf') or filename.endswith('.txt'):
        path = os.path.join("resumes", filename)
        if filename.endswith('.pdf'):
            with open(path, 'rb') as f:
                reader = PyPDF2.PdfReader(f)
                content = ""
                for page in reader.pages:
                    content += page.extract_text()
        else:
            with open(path, 'r') as f:
                content = f.read()
        texts.append(content)
        labels.append(1 if 'good' in filename.lower() else 0)

df = pd.DataFrame({'text': texts, 'label': labels})

vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(df['text'])
y = df['label']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)
model = xgb.XGBClassifier(use_label_encoder=False, eval_metric='logloss')
model.fit(X_train, y_train)

predictions = model.predict(X_test)
print("Accuracy:", accuracy_score(y_test, predictions) * 100)

def check_resume_quality(text):
    vector = vectorizer.transform([text])
    result = model.predict(vector)[0]
    return "Good Resume ✅" if result == 1 else "Poor Resume ❌"

sample = "Completed BTech, interned at Infosys, built AI projects, certified in cloud computing"
print(check_resume_quality(sample))


Saving resume_samples.zip to resume_samples.zip
Accuracy: 50.0
Poor Resume ❌


/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [17:19:29] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
